<a href="https://colab.research.google.com/github/saivenkatreddy29/Implementation-of-Hands-on-Large-Language-models-book/blob/main/Jaylamar_book_chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

data = load_dataset("rotten_tomatoes")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [3]:
data['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [4]:
data['train'][-1]

{'text': 'things really get weird , though not particularly scary : the movie is all portent and no content .',
 'label': 0}

# **Text Classification using representational models**

Using Task specific model

In [5]:
from transformers import pipeline

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe = pipeline(
    model = model_path,
    tokenizer = model_path,
    return_all_scores = True,

)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [6]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_preds = []
for output in tqdm(pipe(KeyDataset(data['test'], 'text')), total = len(data['test'])):

  negative_score = output[0]['score']
  positive_score = output[2]['score']
  # print(negative_score)
  # print(positive_score)
  output = np.argmax([negative_score, positive_score])
  y_preds.append(output)

100%|██████████| 1066/1066 [02:26<00:00,  7.30it/s]


In [7]:
np.unique(data['test']['label'])

array([0, 1])

In [8]:
from sklearn.metrics import classification_report
y_true = data['test']['label']
print(classification_report(y_true, y_preds, target_names=['negative_score', 'positive_score']))

                precision    recall  f1-score   support

negative_score       0.76      0.88      0.81       533
positive_score       0.86      0.72      0.78       533

      accuracy                           0.80      1066
     macro avg       0.81      0.80      0.80      1066
  weighted avg       0.81      0.80      0.80      1066



# **Classification Tasks that leverage Embeddings**

In [9]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 6.7 MB/s eta 0:00:00


In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

train_embeddings = model.encode(data['train']['text'], show_text = True)
test_embeddings = model.encode(data['test']['text'], show_text = True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
len(data['train'])

8530

In [12]:
train_embeddings.shape

(8530, 768)

In [13]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(train_embeddings, data['train']['label'])


LogisticRegression()

In [14]:
y_preds = lr.predict(test_embeddings)

In [15]:
print(classification_report(data['test']['label'], y_preds))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85       533
           1       0.86      0.85      0.85       533

    accuracy                           0.85      1066
   macro avg       0.85      0.85      0.85      1066
weighted avg       0.85      0.85      0.85      1066



**When we don't have the labels we try to find out the Cosine similarity between the sentence in order to get the relation**

In [16]:
label_embeddings = model.encode(['A negative review', ' A positive review'])


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(test_embeddings, label_embeddings)


In [18]:
y_preds = np.argmax(similarity, axis= 1)

In [19]:
print(classification_report(data['test']['label'], y_preds))

              precision    recall  f1-score   support

           0       0.78      0.77      0.78       533
           1       0.77      0.79      0.78       533

    accuracy                           0.78      1066
   macro avg       0.78      0.78      0.78      1066
weighted avg       0.78      0.78      0.78      1066



**experiment by using very negative and positive reviews**

In [20]:
exp_label = model.encode(['A very negative review', 'A very positive review'])


In [21]:
exp_similarity = cosine_similarity(test_embeddings, exp_label)
y_preds = np.argmax(exp_similarity, axis= 1)


In [22]:
print(classification_report(data['test']['label'], y_preds))

              precision    recall  f1-score   support

           0       0.73      0.84      0.78       533
           1       0.81      0.69      0.75       533

    accuracy                           0.76      1066
   macro avg       0.77      0.76      0.76      1066
weighted avg       0.77      0.76      0.76      1066



exp 2

In [23]:
exp_label_2 = model.encode(['A very negative ugly unpleasent and waste review', 'A very good positive pleasent and beautiful review'])


In [24]:
exp_similarity = cosine_similarity(test_embeddings, exp_label_2)
y_preds = np.argmax(exp_similarity, axis= 1)

In [26]:
print(classification_report(data['test']['label'], y_preds))

              precision    recall  f1-score   support

           0       0.67      0.92      0.78       533
           1       0.88      0.55      0.68       533

    accuracy                           0.74      1066
   macro avg       0.77      0.74      0.73      1066
weighted avg       0.77      0.74      0.73      1066



# **Using Open AI's Api's for classification**


In [28]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [29]:
import openai

In [39]:
client = openai.OpenAI(api_key = )

In [40]:
def chatgpt_generation (prompt, document, model = 'gpt-3.5-turbo-0125'):
  """ Generate an output based on a prompt and an input document"""

  messages = [
      {
          'role': 'system',
          'content': 'you are a helpful assistant'
      },
      {
          'role': 'user',
          'content': prompt.replace('[Document]', document)
      }
  ]
  chat_completion = client.chat.completions.create(
      messages = messages,
      model = model,
      temperature = 0
  )
  return chat_completion.choices[0].message.content

In [41]:
prompt = """ Predict wheather the following document is positive or neagative, only give 'Positve' and 'Negative' don't give any other answers: [Document]"""

In [42]:
document = "Unpretentious, charming, quirky, original"

In [44]:
neg_doc = 'How idiotic this movie is'

In [46]:
neu_doc = 'This is an okay movie'

In [48]:
tricky_doc = "It is not okay to unwatch this movie"

In [43]:
chatgpt_generation(prompt, document)

'Positive'

In [45]:
chatgpt_generation(prompt, neg_doc)

'Negative'

In [47]:
chatgpt_generation(prompt, neu_doc)

'Positive'

In [50]:
chatgpt_generation(prompt, tricky_doc)

'Negative'